### Prompting Engineering

In [3]:
%pip install openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [4]:
# 환경설정
import os
from dotenv import load_dotenv

load_dotenv()  # 현재 경로의  .env 파일을 읽어 시스템 환경변수로 등록

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

### Chat Completion

In [8]:
from openai import OpenAI

# clinet-server
client = OpenAI(api_key=OPENAI_API_KEY)

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role":"system", 
            "content":[
                {
                    "type":"text",
                    "text":"당신은 아주 유쾌한 해적 선장입니다."
                }
            ]
        },
        {
            "role":"user", 
            "content":[
                {
                    "type":"text",
                    "text":"추운날 점심에는 뭘 먹으면 좋을까요?"
                }
            ]
        }
    ],
    response_format={ "type": "text" },
    temperature=1.3,    # 대답 창의성(기본값 1) 0~2
    max_tokens= 2048,   # 응답 최대 토큰수
    top_p=1             # 사용할 상위누적확률
)

print(response.choices[0].message.content)



어어! 추운 날씨엔 맛있는 따뜻한 요리가 최고지! 뜨거운 어묵탕이나 순댓국 같은 국물 요리가 몸을 녹이기엔 딱일 거야. 아니면 고추장찌개 같은 알싸한 맛으로 몸을 북돋우는 것도 좋지. 선상에서 겨울바람 맞을 때도 늘 그런 요리로 따뜻하게 곤곤하게 했지. 배를 든든히 채우고 힘차게 하루를 시작해보라구!🏴‍☠️☃️


### 패턴1: 페르소나 & Few-Shot(기사 제목 교정)

In [9]:
from openai import OpenAI

def correct_title(query, temperature=0.3):

    # clinet-server
    client = OpenAI(api_key=OPENAI_API_KEY)

    system_instruction="""
    당신은 신문사의 베테랑 편집장입니다. 기사들이 송고한 제목을 교정해주세요.

    ### 지시사항 ###
    - 기사의 제목이 명확하고 주제와 잘 맞도록 수정해주세요
    - 비속어, 은어등은 제거하고 의미가 유지되도록 제목을 교정해주세요.
    - 간결하고 임팩트 있는 표현을 사용해주세요.

    ### 출력 예시 ###
    - 원래 제목 : [기사의 원래 제목]
    - 교정 제목 :
        [교정된 제목1]
        [교정된 제목2]

    ## 예시 ###
    - 원래 제목 : "어제 서울에서 큰불이 나서 수백명이 대피했다"
    - 교정 제목 : 
        "서울 대형화재, 수백명 대피"
        "수백명이 큰불로 도망!"
    """

    user_message=f"""
    다음 제목을 교정해주세요.
    제목 : {query}
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role":"system", 
                "content":[
                    {
                        "type":"text",
                        "text":system_instruction
                    }
                ]
            },
            {
                "role":"user", 
                "content":[
                    {
                        "type":"text",
                        "text":user_message
                    }
                ]
            }
        ],
        response_format={ "type": "text" },
        temperature=temperature,    # 대답 창의성(기본값 1) 0~2
        max_tokens= 2048,   # 응답 최대 토큰수
        top_p=1             # 사용할 상위누적확률
    )

    return response.choices[0].message.content

print(correct_title('이은지 FM 개꿀 라디오 방송에 주목해주세요'))
print(correct_title('졸라 빡쎈 작업으로 끼니를 거르는 이 시대 노동자의 애환'))


    - 원래 제목 : 이은지 FM 개꿀 라디오 방송에 주목해주세요
    - 교정 제목 : 
        "이은지 FM 라디오 방송, 주목받는 이유"
        "이은지 FM 라디오, 청취자 관심 집중"

    - 원래 제목 : 졸라 빡쎈 작업으로 끼니를 거르는 이 시대 노동자의 애환
    - 교정 제목 :
        "고된 작업에 끼니도 거르는 현대 노동자의 고충"
        "현대 노동자, 과중한 업무로 끼니 거른다"


### 구조화된 출력 (Structured Ouptput) - 영단어장 생성

In [10]:
from openai import OpenAI
import json # API로부터 받은 JSON형식의 텍스트 응답을 파이썬 객체로 변환하는데 사용

def extract_eng_words(query, temperature=0.3):

    # clinet-server
    client = OpenAI(api_key=OPENAI_API_KEY)

    system_instruction="""
    당신은 영어팝송을 이용해 흥미롭고 이해하기 쉬운 방식으로 영어를 가르치는 선생님입니다.

    # 처리단계
    1. 주어진 가사에서 자주 사용되는 영어단더 5개를 랜덤으로 추출해주세요.
    2. 각 단어의 의미를 한글로 알려주세요.
    3. 단어별로 유사한 단어도 함께 소개해주세요.

    # 출력형식
    출력형식은 json 형식입니다.
    - 최상위 키는 "json_list" 여야 합니다.
    -"json_list"의 값은 단어별 json 객체들이 담긴 리스트여야 합니다.

    # 출력형식
    {
        "json_list" : [
            {
                "단어" : "yesterday",
                "의미" : "어제"
                "유사어" : [
                    {
                        "유사단어" : "today"    
                    }, 
                    {
                        "유사단어" : "tomorrow"
                    }
                ]
            }
        ]
    }
    """

    user_message=f"""
    노래가사: {query}
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role":"system", 
                "content":[
                    {
                        "type":"text",
                        "text":system_instruction
                    }
                ]
            },
            {
                "role":"user", 
                "content":[
                    {
                        "type":"text",
                        "text":user_message
                    }
                ]
            }
        ],
        response_format={ "type": "json_object" },
        temperature=temperature,    # 대답 창의성(기본값 1) 0~2
        max_tokens= 2048,   # 응답 최대 토큰수
        top_p=1             # 사용할 상위누적확률
    )


    return json.loads(response.choices[0].message.content) # 응답 json문자열을 파이썬 객체로 변환 

lyrics="Yesterday all my troubles seemed so far away Now it looks as though they're here to stay Oh, I believe in yesterday Suddenly I'm not half the man I used to be There's a shadow hanging over me Oh, yesterday came suddenly Why she had to go I don't know, she wouldn't say I said something wrong now I long for yesterday Yesterday love was such an easy game to play Now I need a place to hide away Oh, I believe in yesterday. Why she had to go I don't know, She wouldn't say I said something wrong Now I long for yesterday Yesterday love was such an easy game to play Now I need a place to hide away Oh, I believe in yesterday"

result = extract_eng_words(lyrics, temperature=1)
words = result['json_list']

# 파이썬 객체로 변환되었기 때문에 for 반복문을 통해 각 요소를 순회하고 딕셔너리의 키를 이용해 원하는 데이터에 직접 접근 가능

for word_dict in words:
    print(f"단어 : {word_dict["단어"]}")
    print(f"의미 : {word_dict["의미"]}")

단어 : yesterday
의미 : 어제
단어 : troubles
의미 : 문제들
단어 : believe
의미 : 믿다
단어 : suddenly
의미 : 갑자기
단어 : love
의미 : 사랑


### 생각의 사슬(Chain-of-Thought) (냉털 마스터)
* Reason + Act 기법으로 형재 상황에 대한 통찰이후, 다음 행동에 대한 작성을 유도하는 기법

In [11]:
from openai import OpenAI
import json # API로부터 받은 JSON형식의 텍스트 응답을 파이썬 객체로 변환하는데 사용

def my_refregator(query, temperature=0.3):

    # clinet-server
    client = OpenAI(api_key=OPENAI_API_KEY)

    system_instruction="""
    너는 냉장고에 있는 재료들을 활용하여 창의적이고 실용적인 저녁식사 아이디어를 제안하는 요리 전문가이다.
    네 역할은 사용자로부터 제공받은 재료목록을 분석하고, 이를 활용할 수 있는 요리 아이디어를 구상해서,
    조리방법을 단계별로 상세히 설명하는 것이다.

    # 출력예시
    1. 상황분석
    - 현재 가진 재료는 [사용자가 입력한 재료] 입니다.
    - 주재료인 닭고기와 다양한 채소들이 있습니다.
    - 이 재료들은 스튜, 볶음, 찜요리에 적합합니다.

    2. 행동계획
    - 가장 쉽게 만들수 있는 요리로 닭볶음탕을 제안합니다.
    - 각 요리에 필요한 재료와 조리도구를 확인합니다.
    - 부족한 재료가 있다면, 대체가능한 옵션을 제시합니다.
    - 조리 과정을 단계별로 상세히 설명합니다.
    - 맛을 더 향상시킬수 있는 팁과 주의사항 또한 제공합니다. 

    3. 실행
    - 여기에 상세레시피를 단계별로 작성합니다.
    """

    user_message=f"""
   사용자의 냉장고에는 현재 {query}가 있습니다.
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role":"system", 
                "content":[
                    {
                        "type":"text",
                        "text":system_instruction
                    }
                ]
            },
            {
                "role":"user", 
                "content":[
                    {
                        "type":"text",
                        "text":user_message
                    }
                ]
            }
        ],
        response_format={ "type": "text" },
        temperature=temperature,    # 대답 창의성(기본값 1) 0~2
        max_tokens= 2048,   # 응답 최대 토큰수
        top_p=1             # 사용할 상위누적확률
    )


    return response.choices[0].message.content # 응답 json문자열을 파이썬 객체로 변환 

print(my_refregator("감자, 소고기, 된장, 표고버섯", temperature=1.2))
    


1. 상황분석
- 현재 가진 재료는 감자, 소고기, 된장, 표고버섯입니다.
- 재료를 고려했을 때, 이들로 된장찌개나 한그릇요리로 활용할 수 있습니다.
- 주재료인 소고기와 된장을 활용하여 감자와 표고버섯을 더한 요리를 만들 수 있습니다.

2. 행동계획
- 소고기와 된장을 활용하여 감자 및 표고버섯을 더한 '소고기 감자 된장찌개'를 제안합니다.
- 사용할 재료와 필요한 도구들을 확인합니다:
  - 소고기, 감자, 된장, 표고버섯, 양파 (옵션)
  - 물, 다진 마늘, 대파 (없으면 제외 가능), 고춧가루 (옵션)
  - 냄비, 도마, 칼
- 만약 냉장고에 물이나 양파, 대파가 없다면, 물은 생수나 육수로 대체 가능합니다. 양파 대파는 없으면 생략 가능합니다.
- 조리 과정을 단계별로 안내합니다.
- 맛을 더하기 위한 작은 팁과 주의사항도 설명합니다.

3. 실행
- 상세레시피:

    1. 재료 준비:
        - 소고기는 먹기 좋은 크기로 썬다.
        - 감자는 껍질을 벗기고 큼직하게 썬다.
        - 표고버섯은 밑둥을 제거하고 적당한 두께로 썬다.
        - 다진 마늘과 대파를 송송 썬다 (선택 사항).
        - 양파를 추가 예정이라면 굵게 채 썬다.

    2. 소고기 양념하기:
        - 소고기는 소금과 후추로 간을 한 후 약간의 다진 마늘을 넣어 잠시 재워둔다.

    3. 찌개 국물 만들기:
        - 냄비에 물이나 육수(약 3컵)를 붓고 중불로 끓이기 시작한다.
        - 육수가 끓어오르면 된장을 적당한 양(약 2 큰술) 섞어 풀어준다.
        - 맛을 조절하고 싶다면 고춧가루를 약간 첨가해 매콤한 맛을 더해도 된다 (선택사항).

    4. 채소와 소고기 끓이기:
        - 된장을 푼 국물이 끓어오르면 감자, 소고기와 양파를 넣고 중불에서 끓인다.
        - 감자가 익을 때까지 약 15분 정도 끓인다.

    5. 표고버섯과 마무리:
        - 

### 면접 질문 생성

In [12]:
from openai import OpenAI
import json # API로부터 받은 JSON형식의 텍스트 응답을 파이썬 객체로 변환하는데 사용

def job_interview(job_postiong, temperature=0.3):

    # clinet-server
    client = OpenAI(api_key=OPENAI_API_KEY)

    system_instruction="""
    당신은 풀스택과 AI개발 분야의 전문가이면서, 해당 분야의 면접 전문가입니다.
    사용자가 제공한 구인공고에 근거해서 핵심적인 질문들로 지원자를 평가할 수 있어야 합니다. 
    """

    user_message=f"""
   아래 채용공고의 직무에 대해 예상면접 질문과 모범답안을 작성해주세요.

   -- 가이드 --
   하드스킬과 소프트스킬 2개의 섹션으로 나눠 작성해 주세요.
   각 섹션별로 2개의 질문과 답변을 준비해주세요.

   -- 출력형식 --
   # 1.  하드스킬
   질문1)
   답변1)

   질문2)
   답변2)

   # 2. 소프트스킬
   질문1)
   답변1)

   질문2)
   답변2)

   -- 채용공고 --
   {job_postiong}
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role":"system", 
                "content":[
                    {
                        "type":"text",
                        "text":system_instruction
                    }
                ]
            },
            {
                "role":"user", 
                "content":[
                    {
                        "type":"text",
                        "text":user_message
                    }
                ]
            }
        ],
        response_format={ "type": "text" },
        temperature=temperature,    # 대답 창의성(기본값 1) 0~2
        max_tokens= 2048,   # 응답 최대 토큰수
        top_p=1             # 사용할 상위누적확률
    )


    return response.choices[0].message.content # 응답 json문자열을 파이썬 객체로 변환 

job_postiong = """

담당업무
- 웹 서비스 운영 및 개발

- 기획 협의 및 화면 구현

- GitLab 기반 소스 관리


자격요건
- 학사 이상

- 경력 : 5년 이내 

- 전공: 컴퓨터공학 및 전산 관련 전공하신 분

- 웹 개발 경험 및 협업 툴 사용 경험이 있으신 분

- 기본적인 코드 리뷰 대응 능력을 보유하신 분

- React, Node.js, Asp.net 활용 가능 하신 분


우대사항
- React, Svelte 프로젝트 경험이 있으신 분

- UI/UX에 대한 이해도가 있으신 분

- 웹 접근성 고려 개발 경험 및 Git 기반 현업 경험이 있으신 분


공통 우대사항
- 주식시장과 기업공시에 대한 기본 지식을 갖추신분

- 빠른 면접진행 및 출근가능자 우대


기타사항
- 긍정 마인드, 책임감, 솔선수범

"""

print(job_interview(job_postiong, temperature=1.2))
    

# 1. 하드스킬

질문1) 
React와 Node.js로 웹 서비스를 개발하면서 마주쳤던 도전적인 문제는 무엇이었으며, 이를 어떻게 해결하셨나요?

답변1) 
한 번은 대규모 데이터를 관리하고 실시간으로 사용자에게 동적 콘텐츠를 제공해야 하는 프로젝트를 진행했습니다. 서버 성능 이슈가 발생했는데, 이에 따라 적절한 비동기 요청이 필요했습니다. React에서는 가상 DOM을 활용하여 컴포넌트 상태 변화를 최적화했으며, Node.js에서는 기능별로 마이크로서비스들로 분리하여 요청을 처리했습니다. 마지막으로 배포 전, Websocket을 통한 데이터 동기화로 실시간 사용자 경험을 최적화했습니다. 이를 통해 성능을 크게 개선할 수 있었습니다.

질문2) 
GitLab을 이용한 소스 관리 경험에 대해 설명하고, 발생할 수 있는 코드 충돌을 어떻게 해결했는지 설명해 주세요.

답변2) 
GitLab을 활용하여 빠르게 기능을 테스트하고 수많은 브랜치를 효과적으로 관리할 수 있습니다. 예전 프로젝트에서 다양한 기능을 병행 개발하다 보니 코드 충돌이 자주 발생했었습니다. 이러한 경우, 일단 결합된 코드 충돌 문제를 GitLab CI/CD 파이프라인의 자동화 테스트를 통해 빠르게 확인하고 수동으로 각 기능의 브랜치를 작성자가 직접 검토하였습니다. 이렇게 코드 리뷰와 문서를 강화하여 팀 내 다른 개발자들과의 원활한 소통을 통해 문제를 해결했습니다.

# 2. 소프트스킬

질문1) 
기획자와 협업하여 화면을 구현하는 과정에서 발생한 의사소통 문제를 해결하신 경험이 있나요?

답변1) 
결제 시스템과 관련된 구성요소를 구현할 때, 기획자가 작성한 요구사항이 구체적이지 않아 문제가 되었습니다. 그래서 먼저 기획자와 추가 미팅을 통해 예산에 맞는 접습 가능한 명확한 요구사항 정의를 했습니다. 또한, UI/UX 시안을 좀 더 명확하게 설계할 수 있도록 공감을 얻어 화면 와이어프레임 초안을 디자이너와 함께 만들어 기획자에게 피드백을 받았습니다. 이러한 절차를 통해 서로의 이해를 도와줄 수